In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 1. 데이터 불러오기
lateralraise = pd.read_csv('./lateralraise_coords.csv')

In [3]:
# 2. 각도 계산 함수
def calculateAngle(a, b, c):
    a = np.array(a)  # 첫 번째 지점
    b = np.array(b)  # 중간 지점
    c = np.array(c)  # 끝 지점

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [4]:
# 3. 관절 각도를 계산하는 함수
def calculate_joint_angles(row):
    nose = [row['x1'], row['y1']]
    left_shoulder = [row['x12'], row['y12']]
    left_elbow = [row['x14'], row['y14']]
    left_wrist = [row['x16'], row['y16']]
    left_hip = [row['x26'], row['y26']]
    left_knee = [row['x28'], row['y28']]
    left_ankle = [row['x30'], row['y30']]
    left_heel = [row['x32'], row['y32']]
    right_shoulder = [row['x13'], row['y13']]
    right_elbow = [row['x15'], row['y15']]
    right_wrist = [row['x17'], row['y17']]
    right_hip = [row['x27'], row['y27']]
    right_knee = [row['x29'], row['y29']]
    right_ankle = [row['x31'], row['y31']]
    right_heel = [row['x33'], row['y33']]

    neck_angle = (calculateAngle(left_shoulder, nose, left_hip) + calculateAngle(right_shoulder, nose, right_hip)) / 2
    left_elbow_angle = calculateAngle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculateAngle(right_shoulder, right_elbow, right_wrist)
    left_shoulder_angle = calculateAngle(left_elbow, left_shoulder, left_hip)
    right_shoulder_angle = calculateAngle(right_elbow, right_shoulder, right_hip)
    left_hip_angle = calculateAngle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculateAngle(right_shoulder, right_hip, right_knee)
    left_knee_angle = calculateAngle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculateAngle(right_hip, right_knee, right_ankle)
    left_ankle_angle = calculateAngle(left_knee, left_ankle, left_heel)
    right_ankle_angle = calculateAngle(right_knee, right_ankle, right_heel)

    return pd.Series([neck_angle, left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_hip_angle, right_hip_angle, left_knee_angle, right_knee_angle, left_ankle_angle, right_ankle_angle])

In [5]:
# 4. 관절 각도 계산 및 데이터프레임에 추가
angle_columns = ['neck_angle', 'left_elbow_angle', 'right_elbow_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'left_hip_angle', 'right_hip_angle', 'left_knee_angle', 'right_knee_angle', 'left_ankle_angle', 'right_ankle_angle']
lateralraise[angle_columns] = lateralraise.apply(calculate_joint_angles, axis=1)

In [6]:
# 5. 각도 데이터를 정규화
scaler = MinMaxScaler()
scaled_angles = scaler.fit_transform(lateralraise[angle_columns])

In [7]:
# 6. 정규화된 데이터를 데이터프레임에 추가
scaled_angles_df = pd.DataFrame(scaled_angles, columns=[f"scaled_{col}" for col in angle_columns])
lateralraise = pd.concat([lateralraise, scaled_angles_df], axis=1)

In [8]:
# 7. 결과 저장
lateralraise.to_csv("lateralraise_with_scaled_angles.csv", index=False)

In [9]:
# 8. 결과 출력
print(lateralraise.head())

   class posture_type        x1        y1        z1        v1        x2  \
0    377      correct  0.514252  0.403630  0.093563  0.999134  0.512110   
1    377      correct  0.514418  0.403220  0.088936  0.999077  0.512114   
2    377      correct  0.514362  0.401675  0.089466  0.999065  0.511837   
3    377      correct  0.515669  0.401734  0.089230  0.999135  0.513123   
4    377      correct  0.518273  0.401750  0.082670  0.999220  0.515514   

         y2        z2        v2  ...  scaled_left_elbow_angle  \
0  0.392551  0.090810  0.998721  ...                 0.966475   
1  0.391994  0.086265  0.998671  ...                 0.979844   
2  0.390314  0.086594  0.998679  ...                 0.989509   
3  0.390372  0.086921  0.998782  ...                 0.981455   
4  0.390416  0.075140  0.998902  ...                 0.365886   

   scaled_right_elbow_angle  scaled_left_shoulder_angle  \
0                  0.981047                    0.004427   
1                  0.978438             